In [1]:
from flask import Flask, request, jsonify
from keras.layers import (Input, Embedding, Flatten, Dot, Dense, Concatenate, Dropout)
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping
import requests
import numpy as np

Using TensorFlow backend.


In [2]:
class DQN:

    def __init__(self, interface):
        # For each sample we input the integer identifiers
        # of a single user and a single item
        user_id_input = Input(shape=(1,), name='user')
        item_id_input = Input(shape=(1,), name='item')
        metadata_input = Input(shape=(interface.nb_variables,), name='metadata')

        embedding_size = 30
        user_embedding = Embedding(output_dim=embedding_size, input_dim=interface.nb_users + 1,
                                   input_length=1, name='user_embedding')(user_id_input)

        item_embedding = Embedding(output_dim=embedding_size, input_dim=interface.nb_items + 1,
                                   input_length=1, name='item_embedding')(item_id_input)

        # reshape from shape: (batch_size, input_length, embedding_size)
        # to shape: (batch_size, input_length * embedding_size) which is
        # equal to shape: (batch_size, embedding_size)
        user_vecs = Flatten()(user_embedding)
        item_vecs = Flatten()(item_embedding)
        # metadata_vecs = Flatten()(metadata_input)

        conc = Concatenate(axis=1)([user_vecs, item_vecs, metadata_input])

        dense_1 = Dense(256, activation='relu')(conc)

        dropout_1 = Dropout(0.5)(dense_1)

        dense_2 = Dense(128, activation='relu')(dropout_1)

        dropout_2 = Dropout(0.5)(dense_2)

        y = Dense(1)(dropout_2)

        self.model = Model(inputs=[user_id_input, item_id_input,
            metadata_input], outputs=y)
        self.model.compile(optimizer='adam', loss='mean_squared_error')
        self.model.save('env2_initial_weight.h5')

    def reset(self):
        self.model = load_model('env2_initial_weight.h5')

    def train(self, user_id_train, item_id_train, metadata, rating_train):
        early_stopping = EarlyStopping(monitor='val_loss', patience=2)
        self.model.fit([user_id_train, item_id_train, metadata], rating_train,
                       batch_size=64, epochs=20, validation_split=0.1,
                       shuffle=True, callbacks=[early_stopping])

    def predict(self, user_id, item_id, metadata):
        return self.model.predict([user_id, item_id, metadata])
    
    def online(self, user_id_train, item_id_train, metadata, rating_train):
        self.model.fit([user_id_train, item_id_train, metadata], rating_train,
                       batch_size=64, epochs=1, validation_split=0.,
                       shuffle=True)

In [3]:
class Interface:

    def __init__(self, args):
        self.base_url = 'http://{}'.format(args.ip_address_env_1)
        self.user_id = args.user_id
        self.url_reset = '{}/reset'.format(self.base_url)
        self.url_predict = '{}/predict'.format(self.base_url)

        r = requests.get(url=self.url_reset, params={'user_id': self.user_id})
        data = r.json()

        self.item_history = data['item_history']
        self.rating_history = data['rating_history']
        self.user_history = data['user_history']
        self.variables_history = data['variables_history']

        self.nb_items = data['nb_items']
        self.nb_users = data['nb_users']
        self.nb_variables = len(data['variables_history'][0])

        self.next_user = data['next_user']
        self.next_item = data['next_item']
        self.next_variables = data['next_variables']

    def reset(self):

        r = requests.get(url=self.url_reset, params={'user_id': self.user_id})
        data = r.json()

        self.item_history = data['item_history']
        self.rating_history = data['rating_history']
        self.user_history = data['user_history']
        self.variables_history = data['variables_history']

        self.nb_items = data['nb_items']
        self.nb_users = data['nb_users']
        self.nb_variables = len(data['variables_history'][0])

        self.next_user = data['next_user']
        self.next_item = data['next_item']
        self.next_variables = data['next_variables']

    def request(self, predicted_score):
        params = {}
        params['user_id'] = self.user_id
        params['predicted_score'] = predicted_score

        r = requests.get(url=self.url_predict, params=params)
        result = r.json()

        next_user = result['next_user']
        next_item = result['next_item']
        next_variables = result['next_variables']
        rating = result['rating']

        return next_user, next_item, next_variables, rating

In [4]:
class Trainer:

    def __init__(self, interface):
        self.interface = interface
        self.dqn = DQN(interface)

    def train(self):
        item_history = self.interface.item_history
        rating_history = self.interface.rating_history
        user_history = self.interface.user_history
        variables_history = self.interface.variables_history
        self.dqn.train(user_history, item_history, variables_history, rating_history)

    def reset(self):
        self.dqn.reset()
        self.train()

    def predict(self, user_id, item_id, metadata):
        return self.dqn.predict(user_id, item_id, metadata).item()
    
    def online(self, user_id, item_id, metadata, rating):
        self.dqn.online(user_id, item_id, metadata, rating)

In [5]:
class Argument:
    pass

args = Argument
args.user_id = 'R3EIFXNYY6XMBXBR01BK'
args.ip_address_env_0 = '52.47.62.31'
args.ip_address_env_1 = '35.180.254.42'
args.ip_address_env_2 = '35.180.178.243'

interface = Interface(args)
trainer = Trainer(interface)
interface.reset()
trainer.reset()

Train on 1800 samples, validate on 200 samples
Epoch 1/20
1800/1800 [==============================] - 1s 298us/step - loss: 4.5976 - val_loss: 2.3522
Epoch 2/20
1800/1800 [==============================] - 0s 74us/step - loss: 2.8397 - val_loss: 2.0976
Epoch 3/20
1800/1800 [==============================] - 0s 83us/step - loss: 2.5918 - val_loss: 1.8468
Epoch 4/20
1800/1800 [==============================] - 0s 79us/step - loss: 1.9694 - val_loss: 1.3882
Epoch 5/20
1800/1800 [==============================] - 0s 81us/step - loss: 1.3550 - val_loss: 1.0992
Epoch 6/20
1800/1800 [==============================] - 0s 81us/step - loss: 1.1183 - val_loss: 1.0243
Epoch 7/20
1800/1800 [==============================] - 0s 82us/step - loss: 0.9698 - val_loss: 0.9872
Epoch 8/20
1800/1800 [==============================] - 0s 72us/step - loss: 0.9961 - val_loss: 0.9064
Epoch 9/20
1800/1800 [==============================] - 0s 87us/step - loss: 0.9373 - val_loss: 0.8925
Epoch 10/20
1800/1800 [==

In [6]:
def update():
    # online training batch
    idx_list = np.random.choice(len(interface.user_history), 63)
    
    user_history = [interface.user_history[i] for i in idx_list]
    variables_history = [interface.variables_history[i] for i in idx_list]
    item_history = [interface.item_history[i] for i in idx_list]
    rating_history = [interface.rating_history[i] for i in idx_list]
    
    user_history.append(interface.next_user)
    variables_history.append(interface.next_variables)
    item_history.append(interface.next_item)
    
    interface.user_history.append(interface.next_user)
    interface.variables_history.append(interface.next_variables)
    interface.item_history.append(interface.next_item)
    
    # get_score
    interface.next_user, interface.next_item, interface.next_variables, rating = interface.request(prediction)
    
    rating_history.append(rating)
    interface.rating_history.append(rating)
    
    # online training
    trainer.online(user_history, item_history, variables_history, rating_history)
    
    return rating

nb_samples = 1000
mse, mae = 0, 0
for i in range(nb_samples):
    # prediction
    prediction = trainer.predict([interface.next_user], [interface.next_item], [interface.next_variables])
    
    # get rating and update the network
    rating = update()
    
    mse += (rating - prediction)**2
    mae += abs(rating - prediction)
print('mse: ', mse/nb_samples)
print('mae: ', mae/nb_samples)

Epoch 1/1
64/64 [==============================] - 0s 80us/step - loss: 0.5986
Epoch 1/1
64/64 [==============================] - 0s 99us/step - loss: 0.7770
Epoch 1/1
64/64 [==============================] - 0s 86us/step - loss: 0.7688
Epoch 1/1
64/64 [==============================] - 0s 76us/step - loss: 0.8382
Epoch 1/1
64/64 [==============================] - 0s 77us/step - loss: 0.6963
Epoch 1/1
64/64 [==============================] - 0s 96us/step - loss: 0.9131
Epoch 1/1
64/64 [==============================] - 0s 78us/step - loss: 0.5785
Epoch 1/1
64/64 [==============================] - 0s 92us/step - loss: 0.6446
Epoch 1/1
64/64 [==============================] - 0s 109us/step - loss: 0.6298
Epoch 1/1
64/64 [==============================] - 0s 94us/step - loss: 1.1934
Epoch 1/1
64/64 [==============================] - 0s 82us/step - loss: 0.7057
Epoch 1/1
64/64 [==============================] - 0s 93us/step - loss: 0.8104
Epoch 1/1
64/64 [==============================] - 

Epoch 1/1
64/64 [==============================] - 0s 99us/step - loss: 0.9255
Epoch 1/1
64/64 [==============================] - 0s 104us/step - loss: 1.1437
Epoch 1/1
64/64 [==============================] - 0s 81us/step - loss: 0.9685
Epoch 1/1
64/64 [==============================] - 0s 73us/step - loss: 0.9262
Epoch 1/1
64/64 [==============================] - 0s 80us/step - loss: 0.6780
Epoch 1/1
64/64 [==============================] - 0s 80us/step - loss: 0.9202
Epoch 1/1
64/64 [==============================] - 0s 80us/step - loss: 0.9417
Epoch 1/1
64/64 [==============================] - 0s 78us/step - loss: 0.4458
Epoch 1/1
64/64 [==============================] - 0s 75us/step - loss: 0.7843
Epoch 1/1
64/64 [==============================] - 0s 90us/step - loss: 0.7068
Epoch 1/1
64/64 [==============================] - 0s 84us/step - loss: 0.5917
Epoch 1/1
64/64 [==============================] - 0s 73us/step - loss: 0.8504
Epoch 1/1
64/64 [==============================] - 

Epoch 1/1
64/64 [==============================] - 0s 66us/step - loss: 0.5461
Epoch 1/1
64/64 [==============================] - 0s 79us/step - loss: 0.7477
Epoch 1/1
64/64 [==============================] - 0s 84us/step - loss: 0.8590
Epoch 1/1
64/64 [==============================] - 0s 66us/step - loss: 0.9124
Epoch 1/1
64/64 [==============================] - 0s 84us/step - loss: 0.7142
Epoch 1/1
64/64 [==============================] - 0s 106us/step - loss: 0.6752
Epoch 1/1
64/64 [==============================] - 0s 83us/step - loss: 0.9597
Epoch 1/1
64/64 [==============================] - 0s 74us/step - loss: 0.7528
Epoch 1/1
64/64 [==============================] - 0s 90us/step - loss: 0.5730
Epoch 1/1
64/64 [==============================] - 0s 79us/step - loss: 0.6983
Epoch 1/1
64/64 [==============================] - 0s 88us/step - loss: 0.7890
Epoch 1/1
64/64 [==============================] - 0s 92us/step - loss: 0.8147
Epoch 1/1
64/64 [==============================] - 

Epoch 1/1
64/64 [==============================] - 0s 76us/step - loss: 0.8059
Epoch 1/1
64/64 [==============================] - 0s 75us/step - loss: 0.5993
Epoch 1/1
64/64 [==============================] - 0s 73us/step - loss: 0.5570
Epoch 1/1
64/64 [==============================] - 0s 97us/step - loss: 1.1405
Epoch 1/1
64/64 [==============================] - 0s 86us/step - loss: 0.7465
Epoch 1/1
64/64 [==============================] - 0s 103us/step - loss: 0.5972
Epoch 1/1
64/64 [==============================] - 0s 93us/step - loss: 0.5186
Epoch 1/1
64/64 [==============================] - 0s 83us/step - loss: 0.6466
Epoch 1/1
64/64 [==============================] - 0s 83us/step - loss: 0.7275
Epoch 1/1
64/64 [==============================] - 0s 82us/step - loss: 0.5277
Epoch 1/1
64/64 [==============================] - 0s 96us/step - loss: 0.6177
Epoch 1/1
64/64 [==============================] - 0s 71us/step - loss: 0.8272
Epoch 1/1
64/64 [==============================] - 

Epoch 1/1
64/64 [==============================] - 0s 101us/step - loss: 0.7011
Epoch 1/1
64/64 [==============================] - 0s 97us/step - loss: 0.6377
Epoch 1/1
64/64 [==============================] - 0s 77us/step - loss: 0.6893
Epoch 1/1
64/64 [==============================] - 0s 69us/step - loss: 0.8457
Epoch 1/1
64/64 [==============================] - 0s 98us/step - loss: 0.8412
Epoch 1/1
64/64 [==============================] - 0s 96us/step - loss: 0.6563
Epoch 1/1
64/64 [==============================] - 0s 103us/step - loss: 0.8435
Epoch 1/1
64/64 [==============================] - 0s 97us/step - loss: 0.5869
Epoch 1/1
64/64 [==============================] - 0s 93us/step - loss: 0.8210
Epoch 1/1
64/64 [==============================] - 0s 74us/step - loss: 0.5900
Epoch 1/1
64/64 [==============================] - 0s 83us/step - loss: 0.7744
Epoch 1/1
64/64 [==============================] - 0s 107us/step - loss: 0.5039
Epoch 1/1
64/64 [==============================] 

Epoch 1/1
64/64 [==============================] - 0s 72us/step - loss: 0.5025
Epoch 1/1
64/64 [==============================] - 0s 99us/step - loss: 0.6881
Epoch 1/1
64/64 [==============================] - 0s 82us/step - loss: 0.7371
Epoch 1/1
64/64 [==============================] - 0s 83us/step - loss: 0.5598
Epoch 1/1
64/64 [==============================] - 0s 104us/step - loss: 0.8834
Epoch 1/1
64/64 [==============================] - 0s 79us/step - loss: 0.7211
Epoch 1/1
64/64 [==============================] - 0s 86us/step - loss: 0.6643
Epoch 1/1
64/64 [==============================] - 0s 110us/step - loss: 0.5461
Epoch 1/1
64/64 [==============================] - 0s 85us/step - loss: 0.8895
Epoch 1/1
64/64 [==============================] - 0s 96us/step - loss: 0.6259
Epoch 1/1
64/64 [==============================] - 0s 96us/step - loss: 0.5854
Epoch 1/1
64/64 [==============================] - 0s 80us/step - loss: 0.6473
Epoch 1/1
64/64 [==============================] -

Epoch 1/1
64/64 [==============================] - 0s 87us/step - loss: 0.9845
Epoch 1/1
64/64 [==============================] - 0s 81us/step - loss: 0.7215
Epoch 1/1
64/64 [==============================] - 0s 85us/step - loss: 0.5810
Epoch 1/1
64/64 [==============================] - 0s 93us/step - loss: 0.5100
Epoch 1/1
64/64 [==============================] - 0s 83us/step - loss: 0.4051
Epoch 1/1
64/64 [==============================] - 0s 80us/step - loss: 0.7110
Epoch 1/1
64/64 [==============================] - 0s 111us/step - loss: 0.4954
Epoch 1/1
64/64 [==============================] - 0s 91us/step - loss: 1.1515
Epoch 1/1
64/64 [==============================] - 0s 92us/step - loss: 0.5007
Epoch 1/1
64/64 [==============================] - 0s 105us/step - loss: 0.7262
Epoch 1/1
64/64 [==============================] - 0s 94us/step - loss: 0.5908
Epoch 1/1
64/64 [==============================] - 0s 71us/step - loss: 0.5759
Epoch 1/1
64/64 [==============================] -

Epoch 1/1
64/64 [==============================] - 0s 88us/step - loss: 0.5050
Epoch 1/1
64/64 [==============================] - 0s 93us/step - loss: 0.6210
Epoch 1/1
64/64 [==============================] - 0s 84us/step - loss: 0.5828
Epoch 1/1
64/64 [==============================] - 0s 83us/step - loss: 0.3930
Epoch 1/1
64/64 [==============================] - 0s 88us/step - loss: 0.5521
Epoch 1/1
64/64 [==============================] - 0s 96us/step - loss: 0.6258
Epoch 1/1
64/64 [==============================] - 0s 76us/step - loss: 0.4063
Epoch 1/1
64/64 [==============================] - 0s 91us/step - loss: 0.8979
Epoch 1/1
64/64 [==============================] - 0s 80us/step - loss: 0.6804
Epoch 1/1
64/64 [==============================] - 0s 81us/step - loss: 0.4602
Epoch 1/1
64/64 [==============================] - 0s 83us/step - loss: 0.7753
Epoch 1/1
64/64 [==============================] - 0s 109us/step - loss: 0.7714
Epoch 1/1
64/64 [==============================] - 

Epoch 1/1
64/64 [==============================] - 0s 114us/step - loss: 0.8159
Epoch 1/1
64/64 [==============================] - 0s 92us/step - loss: 0.5707
Epoch 1/1
64/64 [==============================] - 0s 87us/step - loss: 0.6535
Epoch 1/1
64/64 [==============================] - 0s 91us/step - loss: 0.7098
Epoch 1/1
64/64 [==============================] - 0s 92us/step - loss: 0.6563
Epoch 1/1
64/64 [==============================] - 0s 93us/step - loss: 0.4865
Epoch 1/1
64/64 [==============================] - 0s 92us/step - loss: 0.6324
Epoch 1/1
64/64 [==============================] - 0s 76us/step - loss: 0.6435
Epoch 1/1
64/64 [==============================] - 0s 97us/step - loss: 0.7463
Epoch 1/1
64/64 [==============================] - 0s 95us/step - loss: 0.6377
Epoch 1/1
64/64 [==============================] - 0s 105us/step - loss: 0.6263
Epoch 1/1
64/64 [==============================] - 0s 101us/step - loss: 0.6656
Epoch 1/1
64/64 [==============================] 

Epoch 1/1
64/64 [==============================] - 0s 68us/step - loss: 0.5500
Epoch 1/1
64/64 [==============================] - 0s 82us/step - loss: 0.4377
Epoch 1/1
64/64 [==============================] - 0s 91us/step - loss: 0.4791
Epoch 1/1
64/64 [==============================] - 0s 74us/step - loss: 0.4209
Epoch 1/1
64/64 [==============================] - 0s 93us/step - loss: 0.6480
Epoch 1/1
64/64 [==============================] - 0s 101us/step - loss: 0.7290
Epoch 1/1
64/64 [==============================] - 0s 75us/step - loss: 0.5561
Epoch 1/1
64/64 [==============================] - 0s 84us/step - loss: 0.6715
Epoch 1/1
64/64 [==============================] - 0s 76us/step - loss: 0.6294
Epoch 1/1
64/64 [==============================] - 0s 84us/step - loss: 0.6834
Epoch 1/1
64/64 [==============================] - 0s 75us/step - loss: 0.6780
Epoch 1/1
64/64 [==============================] - 0s 83us/step - loss: 0.5640
Epoch 1/1
64/64 [==============================] - 